In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sqlite3

connection = sqlite3.connect("file:db/words.db?mode=ro")

In [ ]:
username = 'stormalinblue'
user_id = connection.execute('select id from users where users.user_name = ?', (username,)).fetchone()[0]

In [ ]:
import lib.game.model as gamemodel
from lib.common.util import utc_now_sec_timestamp

In [ ]:
import pandas as pd
import numpy as np
import scipy

In [ ]:
prior_correct = 1
prior_incorrect = 1
prior_total = prior_correct + prior_incorrect

word_weight_table = gamemodel.word_weight_table(
    connection,
    user_id,
    utc_now_sec_timestamp(),
    prior_correct=prior_correct,
    prior_incorrect=prior_incorrect)
word_pos_definitions = pd.read_sql(
    '''
select
    word_pos.id as word_pos_id, words.word as word, parts_of_speech.name as pos
from
    word_parts_of_speech as word_pos
    join words on words.id = word_pos.word_id
    join parts_of_speech on parts_of_speech.id == word_pos.part_of_speech_id
order by words.word, parts_of_speech.name''',
    con=connection,
    index_col=['word_pos_id']
)

word_table = pd.concat([word_pos_definitions, word_weight_table], axis=1)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x_axis = np.linspace(0, 1, 100)

n_highest = 5
word_table['mean_prob'] = (word_table['correct']) / (word_table['correct'] + word_table['incorrect'])
highest = word_table.sort_values('mean_prob', ascending=False).head(n_highest)
for word_pos_id, row in highest.iterrows():
    ax.plot(x_axis, scipy.stats.beta(row.correct, row.incorrect).pdf(x_axis), label=f'{row.word} ({row.pos})')

ax.legend()
ax.set_title(f'PDFs for beta distributions for the top {n_highest} words')

In [ ]:
fig, ax = plt.subplots()
lowest = word_table.sort_values('mean_prob', ascending=True).head(n_highest)
for word_pos_id, row in lowest.iterrows():
    ax.plot(x_axis, scipy.stats.beta(row.correct, row.incorrect).pdf(x_axis), label=f'{row.word} ({row.pos})')

ax.legend()
ax.set_title(f'PDFs for beta distributions for the bottom {n_highest} words')

In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel('Incorrect')
ax.set_ylabel('Correct')
ax.set_title("Distribution of 'correct' and 'incorrect' weights for all words")
ax.grid(True)
ax.hist2d(word_table['incorrect'] - prior_incorrect, word_table['correct'] - prior_correct)
ax.set_aspect(1)

In [ ]:
fig, ax = plt.subplots()

ax.hist(word_table['correct'] + word_table['incorrect'] - prior_total, range=(0, 30), bins=60)
ax.set_title('Histogram of weights caused by user over word-pos\'s')
ax.set_xlabel('Weights')
ax.set_ylabel('Num words')

In [ ]:
fig, ax = plt.subplots()

ax.hist(word_table['mean_prob'], range=(0, 1), bins=20)
ax.set_title('Distribution of mean probabilities')
ax.set_xlabel('Mean probability of correctness')
ax.set_ylabel('Number of words')

In [ ]:
overall_correct = (word_table['correct']).sum() / word_table.shape[0]
overall_incorrect = (word_table['incorrect']).sum() / word_table.shape[0]

fig, ax = plt.subplots()
x_axis = np.linspace(0, 1, 1000)
ax.plot(x_axis, scipy.stats.beta(overall_correct, overall_incorrect).pdf(x_axis))

mean_prob = overall_correct / (overall_correct + overall_incorrect)
ax.axvline(mean_prob, linestyle='--')
ax.annotate(
    text=f'$\\mu$ = {mean_prob:.02f}',
    xy=(mean_prob, 0.9),
    xycoords=('data', 'axes fraction'),
    xytext=(10, 0),
    textcoords=('offset points'))
ax.annotate(
    text=f'$\\alpha$ = {overall_correct:.02f}, $\\beta$ = {overall_incorrect:.02f}',
    xy=(0, 0.9),
    xycoords=('data', 'axes fraction'))